<a href="https://colab.research.google.com/github/LucasOsco/RemoteSensingAI/blob/main/SegmentGeospatial_BoundingBox_vShared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Connect to Drive***

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

***Install both dependencies***

In [ ]:
pip install segment-geospatial leafmap localtileserver

In [ ]:
pip install pycrs

***Import libraries***

In [ ]:
import os
import leafmap
import torch
import cv2
import geopandas as gpd
from samgeo import SamGeo, SamGeoPredictor, tms_to_geotiff, get_basemaps
from segment_anything import sam_model_registry
from pathlib import Path

***Load image and shapefile from Drive***

In [ ]:
# Read the image
image = 'gdrive/My Drive/Colab Notebooks/Image/roi_image.tif' # Switch to your directory instead

# Read the shapefile
shapefile = 'gdrive/My Drive/Colab Notebooks/Shape/roi_bbox.shp' # Switch to your directory instead
gdf = gpd.read_file(shapefile)

# Extract bounding boxes from the shapefile
clip_boxes = [list(geom.bounds) for geom in gdf.geometry]

***Create an interactive map***

In [ ]:
zoom = 19
style_bbox = {'color': '#FFFF00', 'weight': 1, 'fillColor': '#FFFF00', 'fillOpacity': 0.1,}
style_segmented = {'color': '#a37aa9', 'weight': 1, 'fillColor': '#a37aa9', 'fillOpacity': 0.9,}

m = leafmap.Map(center=[-22.17615, -51.253043], zoom=zoom)

m.add_raster(image, layer_name='Image')
m.add_vector(shapefile, layer_name='Vector', style=style_bbox)
m

***Initialize SamGeoPredictor class***

In [ ]:
# Load the model from Drive
out_dir = os.path.join('gdrive/My Drive/Colab Notebooks/Model/') # Switch to your directory instead
checkpoint = os.path.join(out_dir, 'sam_vit_h_4b8939.pth')

In [ ]:
img_arr = cv2.imread(image)
model_type = "vit_h"
sam = sam_model_registry[model_type](checkpoint=checkpoint)
predictor = SamGeoPredictor(sam)
predictor.set_image(img_arr)

***Loop SAM through each polygon feature and load the results on the map***

In [ ]:
combined_gdf = gpd.GeoDataFrame()

# Process each clip_box
for i, clip_box in enumerate(clip_boxes):
    masks, _, _ = predictor.predict(src_fp=image, geo_box=clip_box)

    masks_img = f"preds_{i}.tif"
    predictor.masks_to_geotiff(image, masks_img, masks.astype("uint8"))

    vector = f"feats_{i}.geojson"
    temp_gdf = predictor.geotiff_to_geojson(masks_img, vector, bidx=1)
    combined_gdf = combined_gdf.append(temp_gdf)
    combined_gdf.set_geometry('geometry', inplace=True)

    m.add_vector(vector, layer_name=f'Vector_{i}', style=style_segmented)

***Export the combined shapefile to Drive***

In [ ]:
# Set the CRS for the combined GeoDataFrame (use the same CRS as your input shapefile)
combined_gdf.crs = 'EPSG:4326'

# Save the combined GeoDataFrame as a Shapefile
output_shapefile = 'gdrive/My Drive/Colab Notebooks/Output/roi_segmented.shp' # Switch to your directory instead
combined_gdf.to_file(output_shapefile)